In [ ]:
import smtplib, ssl
from datetime import datetime
import smtplib
import time
import imaplib
import email
import traceback 
from pdb import set_trace
import mailparser

In [154]:
class EmailHandler:
    '''
    Email handler to send and recieve emails
    '''
    
    def __init__(self):
        print('Setting the credentials now...')
        self.set_credentials()
    
    def set_credentials(
        self,
        sender_email: str = 'brahmidarling69@gmail.com',
        reciever_email: str = 'testemail2923@gmail.com',
        sender_password: str = r'#Brahmi69'
    ):
        self.sender_email = sender_email
        self.reciever_email = reciever_email
        self.sender_password = sender_password
    
    
    def send_email(
        self,
        message: str = ''
    ):
        print('Sending email now...')
        port = 465  # For SSL

        # Create a secure SSL context
        context = ssl.create_default_context()

        with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
            server.login(self.sender_email, self.sender_password)
            # TODO: Send email here

            server.sendmail(self.sender_email, self.reciever_email, message)

        print(f'Email with message: {message} has been sent to {self.reciever_email} from {self.sender_email}')
    
    def send_frq(
        self,
        frequency = 0.00277778,
        last_sent = None
    ):
        def has_been_freq(
            last_sent: datetime
        ):
            if not last_sent:
                return True
            if (datetime.now() - last_sent).total_seconds() >= frequency:
                return True
            return False

        while True:
            if has_been_freq(last_sent):
                #send_email(message = f'This email has been sent at {datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}')
                self.send_email(message = 'Enter your temperature and oxygen saturation')
                temperature, oxy_satch, time = self.preprocess_email_content(self.read_email())
                # if time not in db:
                #    store it in db
                #
                last_sent = datetime.now()
                
                
    def read_email(
        self
    ):
        '''
        Source: https://codehandbook.org/how-to-read-email-from-gmail-using-python/
        https://stackoverflow.com/questions/17874360/python-how-to-parse-the-body-from-a-raw-email-given-that-raw-email-does-not
        '''
        try:
            mail = imaplib.IMAP4_SSL(SMTP_SERVER)
            mail.login(self.sender_email, self.sender_password)
            mail.select('inbox')

            data = mail.search(None, 'ALL')
            mail_ids = data[1]
            id_list = mail_ids[0].split()   
            first_email_id = int(id_list[0])
            latest_email_id = int(id_list[-1])

            data = mail.fetch(str(latest_email_id), '(RFC822)' )
            for response_part in data:
                arr = response_part[0]
                if isinstance(arr, tuple):
                    msg = email.message_from_string(str(arr[1],'utf-8'))
                    email_subject = msg['subject']
                    email_datetime = dateparser.parse(msg['date'])

                    if msg.is_multipart():
                        for payload in msg.get_payload()[:1]:
                            return payload.get_payload(), email_datetime
                    else:
                        return msg.get_payload(), email_datetime

                    email_from = msg['from']

        except Exception as e:
            traceback.print_exc() 
            print(str(e))


    def preprocess_email_content(
        self,
        email
    ):
        for index, char in enumerate(email[0]):
            if char == '\n':
                temp, o2 = email[0][:index-1].split(',')
                return temp, o2, email[1]